In [28]:
import math; import random
import json
import pandas as pd
import numpy as np

from dataclasses import dataclass
from basenji import dataset, seqnn, trainer, dna_io

import tensorflow as tf
import tensorlayer as tl
        
#==============================================================================#
#                        DATA DEFINITION :: SEQUENCE/HI-C                      #
#==============================================================================#
    
@dataclass 
class seq_hic_set:
    """Represents pairs of linear genomes and resultant Hi-C matrices 
       to train the adversarial networks.""""
    
    # TODO: Represent sequence/Hi-C pairs for training, validation
    
    # 
    
    
#==============================================================================#
#                         GENERATOR NETWORK (AKITA HEAD)                       #
#==============================================================================#
        
class akitaGAN_gen:
    """Generator network"""
    
    def __init__(self, t_image=None, learn_rate : float=1e-4) -> None:
        """Initialize ConvNet parameters for Generator."""
        
        print('[G] Initializing generator...')
        self.w = tf.random_normal_initializer(stddev=0.02)
        self.b = None
        self.g = tf.random_normal_initializer(1., 0.02)
        
        self.lrelu = lambda x : tl.act.lrelu(x, 0.2)
        self.learn_rate = learn_rate
        print('[G] Done!')
        
        print('[G] Loading generator data...')
        self.load_data()
        print('[G] Done!')
        
    def load_data(self) -> None:
        """Load model parameters and sequence data."""
        
        print('[G] Loading model parameters...')
        with open(params_file) as params_open:
            params = json.load(params_open)
        self.params_model = params['model']
        self.params_train = params['train']
        print('[G] Done!')
        
        print('[G] Loading data...')
        tfr_train_full = '%s/tfrecords/%s' % (data_dir, options.tfr_train_pattern)
        train_data = dataset.SeqDataset(tfr_train_full,
        params_train['batch_size'],
        data_stats['seq_length'],
        data_stats['target_length'],
        tf.estimator.ModeKeys.TRAIN)
        
        tfr_eval_full = '%s/tfrecords/%s' % (data_dir, options.tfr_eval_pattern)
        eval_data = dataset.SeqDataset(tfr_eval_full,
        params_train['batch_size'],
        data_stats['seq_length'],
        data_stats['target_length'],
        tf.estimator.ModeKeys.EVAL)
        print('[G] Done!')
        
        
    def build(self) -> None:
        
        print('[G] Constructing generator architecture...')
        # Construct the model
        self.network = seqnn.SeqNN(self.params_model)
        
        # Create trainer module for model
        self.trainer = trainer.Trainer(params_train, train_data, 
                                    eval_data, options.out_dir)
        
        # Compile the model
        self.trainer.compile(self.network)
        
        print('[G] Done!')
        
        
    def vars(self):
        # Returns the Tensorlayer variables with names
        return tl.layers.get_variables_with_name('gen', True, True)
        
    def optimize(self) -> None:
        """Uses an Adam Optimizer to minimize Generator loss."""
        
        adam = tf.train.AdamOptimizer(lr_v, beta1=beta1)
        adam.minimize(self.loss(), var_list=self.vars())
    
#==============================================================================#
#                            DISCRIMINATOR NETWORK                             #
#==============================================================================#
        
class akitaGAN_disc:
    """The architecture for this Discriminator network is largely adapted from
       Qiao Liu et al.'s 'hicGAN' under the MIT License.
    
       See https://github.com/kimmo1019/hicGAN"""
    
    def __init__(self, t_image=None, learn_rate=1e-4) -> None:
        # Initialize ConvNet parameters for Discriminator
        print('[D] Initializing discriminator...')
        self.w = tf.random_normal_initializer(stddev=0.02)
        self.b = None
        self.g = tf.random_normal_initializer(1., 0.02)
        
        self.lrelu = lambda x : tl.act.lrelu(x, 0.2)
        self.t_image = t_image
        self.learn_rate = learn_rate
        print('[D] Done!')
        
    def build(self, is_train=False, reuse=False):
        # Construct the TF layers of the network
        print('[D] Constructing discriminator architecture...')
        
        with tf.variable_scope("dis", reuse=reuse) as vs:
            n = InputLayer(t_image, name='in')
            n = Conv2d(n, 64, (3, 3), (1, 1), act=lrelu, padding='SAME', W_init=self.w, name='n64s1/c')
            
            for i in range(1, 6):
                if i%2 == 1: dim = 1
                else: dim = 2
                
                n = Conv2d(n, 64, (3, 3), (dim, dim), act=lrelu, padding='SAME', W_init=self.w, b_init=self.b, name='n64s2/c'+str(i))
                n = BatchNormLayer(n, is_train=self.is_train, gamma_init=self.g, name='n64s2/b'+str(i))

            n = Conv2d(n, 128, (3, 3), (1, 1), act=lrelu, padding='SAME', W_init=self.w, b_init=self.b, name='n128s1/c')
            n = BatchNormLayer(n, is_train=self.is_train, gamma_init=self.g, name='n128s1/b')

            # Flatten into dense layers
            n = FlattenLayer(n, name='f')
            n = DenseLayer(n, n_units=512, act=lrelu, name='d512')
            n = DenseLayer(n, n_units=1, name='out')
        
        self.network = n
        print('[D] Done!')
        
    #def loss() -> float:
        # Returns the total discriminator loss
        
    def vars(self):
        # Returns the Tensorlayer variables with names
        return tl.layers.get_variables_with_name('dis', True, True)
        
    def optimize(self) -> None:
        # Uses an Adam Optimizer to minimize Generator loss
        adam = tf.train.AdamOptimizer(lr_v, beta1=beta1)
        adam.minimize(self.loss(), var_list=self.vars())
        
    
    

        logits = n.outputs
        n.outputs = tf.nn.sigmoid(n.outputs)
    
#==============================================================================#
#                                  MAIN METHOD                                 #
#==============================================================================#
    
def main():
    
    # Create placeholder Tensors to construct network layers
    t_image = tf.placeholder('float32', [None, 40, 40, 1], name='input_to_generator')
    t_target_image = tf.placeholder('float32', [None, 40, 40, 1], name='t_target_hic_image')
    
    # Instantiate adversarial networks
    generator = akitaGAN_gen(t_image)
    discriminator = akitaGAN_disc(t_target_image)
    
    # Build ConvNet layers
    generator.load_data()
    generator.build()
    discriminator.build()
    
    
    
main()

SyntaxError: EOL while scanning string literal (<ipython-input-28-df80e8471224>, line 19)